In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
import time
import re

from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager


# use GeckoDriver manager to access firefox page via gecko
executable = FirefoxService(GeckoDriverManager().install())

driver = webdriver.Firefox(service=executable)

[WDM] - Downloading: 16.9kB [00:00, 16.7MB/s]                                                                          
[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 1.59M/1.59M [00:00<00:00, 10.4MB/s]


In [2]:
url_base = "https://www.indeed.com/jobs?q="

position_type = ['software engineer', 'full time']
search = '+'.join(' '.join(position_type).split())
search

url_search = url_base + search

In [3]:
all_titles = []
all_companies = []
all_loc = []
all_salary = []
all_type = []
all_text = []

page = driver.get(url_search)   
driver.implicitly_wait(10)

pg = 0

# pull job cards, excluding spacers and final blank card
to_click = [1,2,3,4,5,7,8,9,10,11,13,14,15,16,17]

In [4]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")

In [5]:
while pg < 3:
    
    time.sleep(5)
    # sort by date
    try:
        date_sort = driver.find_element(By.CSS_SELECTOR, ".jobsearch-DesktopSort > span:nth-child(2) > a:nth-child(2)")
        date_sort.click()
        driver.implicitly_wait(10)
    except:
        pass

    for card in to_click:
        time.sleep(2)
        
        card_search = ".jobsearch-ResultsList > li:nth-child({})".format(card) 
        job_card = driver.find_element(By.CSS_SELECTOR,card_search)
        
        try:
            job_card.click()
        
        except ElementClickInterceptedException:
            x_out = driver.find_element(By.CSS_SELECTOR, ".icl-CloseButton")
            x_out.click()
            job_card.click()
            
        elem = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID,"jobsearch-ViewjobPaneWrapper")) #This is a dummy element
        )
#         driver.implicitly_wait(5)

        desc = driver.find_element(By.CLASS_NAME, "jobsearch-jobDescriptionText")
        job_text = desc.text
        all_text.append(job_text)

        soup = BeautifulSoup(job_card.get_attribute('innerHTML'), 'html.parser')

        # get html from root node
        soup_children = list(soup.children)
        html = soup_children[0]    

        # Get head from html 
        html_children = list(html.children)
        head = html_children[0]

        # get job card stats
        job_title = head.find('h2').text
        company = head.find("span", {"class":"companyName"}).text
        location = head.find("div",{"class":"companyLocation"}).text

        attr = head.find("div",{"class":"attribute_snippet"}).text
        if sum(map(str.isdigit, attr)) > 2:
            salary = attr
        else:
            salary = 'n/a'

        all_titles.append(job_title)
        all_companies.append(company)
        all_loc.append(location)
        all_salary.append(salary)
        
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        
    next_page = driver.find_element(By.CSS_SELECTOR, '[data-testid="pagination-page-next"]')
    
    
    try:
        next_page.click()
        
    except ElementClickInterceptedException:
        x_out = driver.find_element(By.CSS_SELECTOR, ".icl-CloseButton")
        x_out.click()
        next_page.click()
#     next_page.click()

    pg += 1
    driver.implicitly_wait(10)
      

In [6]:
criteria = ['title','company','location','salary','description']
df = pd.DataFrame(columns=criteria)

df['title'] = all_titles
df['company'] = all_companies
df['location'] = all_loc
df['salary'] = all_salary
df['description'] = all_text

df

,title,company,location,salary,description
0,Software Engineer,Recruiting From Scratch,"Remote in Hoboken, NJ 07030+126 locations","$130,000 - $170,000 a year",Who is Recruiting from Scratch:\nRecruiting fr...
1,Game Developer,"Andersen Industries, Inc.",Remote,"$73,168 - $250,000 a year",We are looking for Co-Founders who are passion...
2,Full Stack Engineer 1,Humana,"Remote in Louisville, KY 40202+8 locations",n/a,The Full Stack Engineer 1 Performs software en...
3,SOFTWARE ENGINEER I,"Phoenix Logistics, LLC","Hybrid remote in Orlando, FL 32826",n/a,"At Phoenix Defense, our team members have incr..."
4,Game Producer,"Andersen Industries, Inc.",Remote,"$73,168 - $250,000 a year",We are looking for Co-Founders who are passion...
5,Software Engineer,TekwissenGroup,"Milan, IL 61264",$70 - $75 an hour,Overview:\nTekWissen Group is a workforce mana...
6,Software Engineer,Teoresi Group,"Schaumburg, IL 60173+9 locations",n/a,We are looking for an:\nAutomotive Software En...
7,SQL Database Developer,Guardian Mineral Management,Remote,$50 - $80 an hour,· Seeking an intermediate/expert level Databas...
8,Advanced Software Development Engineer,3M,"Pittsburgh, PA+2 locations",n/a,Job Description:\nAdvanced Software Developmen...
9,Software Engineer,Prolim Corporation,"Sterling Heights, MI 48310+3 locations",n/a,Job description: US Citizen Only\nThis is an o...


In [ ]:
# get a subset of tweets using search term
def subdf(df,term):  
    sens = []
    for i in df.index:
        sen = df['full text'][i]
    
        if term in sen.lower():
            sens.extend([i])
            
    return df.loc[sens]

In [ ]:
# clean the descriptions pulled from indeed
def clean_jobcard(x):
    # ignore irrelevant frequent words
    stop_words = ['says','like','said']
    # make regex for components to clean from strings
    regex = "(@([A-Za-z0-9._-]+))|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|([()!?])|(\[.*?\])|(#[A-Za-z0-9_]+)"
    # remove non-alphanumeric chars, punctuation, hashtags/mentions, and links then split into list
    clean=' '.join(re.sub(regex," ",x).split())
    # remove digits
    clean = re.sub(r'\d+', '', clean)
    # only keep non-stop words
    clean = [w for w in clean if w not in stop_words]
    # join words back together into a sentence
    clean = ''.join(word for word in clean) 
    return clean

In [ ]:
# clean all job descriptions in df
df["cleaned_description"]=df.iloc[:,4].apply(clean_jobcard)

In [ ]:
def vec(x):
    vectorizer = CountVectorizer(ngram_range=(1,1), min_df=2, stop_words="english", max_features=50) # default unigram model
    X = vectorizer.fit_transform(x)
    feature_names = vectorizer.get_feature_names_out()
    return X, feature_names

In [ ]:
def cov_mat(X):
    # get the feature to feature co-occurrence matrix
    co = (X.T * X) # this is co-occurrence matrix in sparse csr format
    co.setdiag(0) # sometimes you want to fill same word cooccurence to 0

    # select a threshold to reduce the density of the network
    threshold = 1
    co_matrix = np.where(co.todense()>threshold, 1, 0)
    return co_matrix

In [ ]:
# build the function to analyze semantic network
def build_semantic_network(x):
    df = pd.Series(list(x))
   # get features and adjacency list
    X, feats = vec(df)
    # get feature to feature co-occurrence matrix
    co_matrix = cov_mat(X)
    
    # Build a network out of this co-occurrence matrix
    G = nx.from_numpy_matrix(co_matrix, create_using=nx.Graph()) # creating from adjacency matrix 
    node_name = dict(zip(range(0,len(list(G.nodes()))), feats))  
    G = nx.relabel_nodes(G, node_name)
    return G

In [ ]:
## taken from class slides, subbed eigenvector centrality
def show_semantic_network(G, title):
    # Take a look
    figure(figsize=(20,20), dpi=100)

    # node size proportional to eigenvector centrality of words
    node_size= [x*1000 for x in nx.eigenvector_centrality(G).values()]

    # choose a layout function
    pos=nx.kamada_kawai_layout(G)

    # edge color
    edge_color='0.8'

    nx.drawing.nx_pylab.draw_networkx(G,
                                      node_size=node_size, 
                                      node_color = "pink",
                                      pos=pos,
                                      edge_color=edge_color,
                                      with_labels=True,
                                     font_size=12)
    plt.title(title)
    plt.axis("off") 
    plt.show()

In [ ]:
# show semantic network
G = build_semantic_network(set(df["cleaned_description"]))
show_semantic_network(G, title)